In [ ]:
"""
इनपुट: आप आज कैसा महसूस कर रहे हैं?
आउटपुट: मैं आज बहुत अच्छा महसूस कर रहा हूँ, नए दिन का स्वागत करने को तैयार हूँ।

इनपुट: आप आम तौर पर कौन सा खेल खेलना पसंद करते हैं?
आउटपुट: मुझे आम तौर पर दौड़ना और बास्केटबॉल खेलना पसंद है।

इनपुट: आपका पसंदीदा चीनी व्यंजन क्या है?
आउटपुट: मेरा पसंदीदा व्यंजन मापो टोफू है, क्योंकि यह तीखा और सुगंधित होता है।

इनपुट: आप आम तौर पर कितने बजे उठते हैं?
आउटपुट: मैं आम तौर पर सुबह सात बजे उठता हूँ।

इनपुट: आज आपके काम/अध्ययन का मुख्य कार्य क्या है?
आउटपुट: आज मुझे एक महत्वपूर्ण परियोजना प्रस्तुति पूरी करनी है।

इनपुट: आप किस तरह का संगीत सुनना पसंद करते हैं?
आउटपुट: मुझे पॉप संगीत और हल्की लोक‑गाने सुनना पसंद है।

इनपुट: आप अपनी अगली छुट्टी में कहाँ यात्रा करना चाहेंगे?
आउटपुट: मैं युन्नान जाना चाहूँगा, वहाँ का नजारा देखने तथा स्थानीय व्यंजन चखने के लिए।

इनपुट: क्या आपके पास कोई पालतू है? उसका नाम क्या है?
आउटपुट: मेरे पास एक बिल्ली है, नाम “छोटू” है।

इनपुट: आपकी पसंदीदा फिल्म कौन सी है?
आउटपुट: मेरी पसंदीदा फिल्म “द शॉशैंक रिडेम्प्शन” है, क्योंकि इसकी कहानी बहुत मार्मिक है।

इनपुट: आपके दीर्घकालिक लक्ष्य या सपने क्या हैं?
आउटपुट: मैं दुनिया भर की यात्रा करना और विभिन्न संस्कृतियों का अनुभव करना चाहता हूँ।
"""


In [ ]:
# MORNING CODE

import torch
import torch.nn as nn
import torch.nn.functional as F

class PrunedLlama4TextMoe(nn.Module):
    def __init__(self, original_moe_layer, kept_indices):
        super().__init__()
        self.shared_expert = original_moe_layer.shared_expert
        self.act_fn       = original_moe_layer.experts.act_fn

        # Keep the full router, but remember which experts to mute
        self.router       = original_moe_layer.router
        self.num_experts  = self.router.out_features

        # Build a boolean mask: True for kept, False for pruned
        mask = torch.zeros(self.num_experts, dtype=torch.bool, 
                           device=self.router.weight.device)
        mask[kept_indices] = True
        self.register_buffer('kept_mask', mask)

    def forward(self, x):
        # x: [B, S, H]
        B, S, H = x.shape
        x_flat = x.view(-1, H)                 # [B*S, H]

        # 1) compute full logits: [B*S, E]
        logits_full = self.router(x_flat)

        # 2) mask out the pruned experts by setting their logits to -inf
        #    so their softmax probability becomes zero:
        masked_logits = logits_full.masked_fill(~self.kept_mask, float('-inf'))

        # 3) softmax over all E experts (pruned ones get zero prob)
        dispatch_probs = F.softmax(masked_logits, dim=-1)  # [B*S, E]

        # 4) run the shared expert once
        expert_out = self.shared_expert(x_flat)            # [B*S, H]
        expanded   = expert_out.unsqueeze(1)               # [B*S, 1, H]
        expanded   = expanded.expand(-1, self.num_experts, -1)  # [B*S, E, H]

        # 5) weight & sum across the expert dimension
        out_flat = (dispatch_probs.unsqueeze(-1) * expanded).sum(dim=1)  # [B*S, H]
        out      = out_flat.view(B, S, H)

        # return the final MoE output plus the original-shaped logits
        return out, logits_full
